Remember, while the name "tool calling" implies that the model is directly performing some action, this is actually not the case! The model only generates the arguments to a tool, and actually running the tool (or not) is up to the user.

大模型实际只是“发号施令”，实际执行工具是系统的事情！

langchain 提到提供 tools 给大模型可以有四种方式：
* Python functions (with typehints and docstrings)
* Pydantic models
* TypedDict classes
* LangChain Tool objects

In [1]:
# Python functions

# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

In [ ]:
# LangChain Tool

In [ ]:
# Pydantic class
# Note that all fields are required unless provided a default value.

from pydantic import BaseModel, Field

class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [ ]:
# TypedDict class
# 不推荐这种方式吧，看着不得劲

关于 tools 的生成，再参考下 **custom_tools.ipynb**

可以使用 `.bind_tools()` 把 tool schemas 绑定到大模型。

In [3]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env",verbose=True)

llm = ChatOpenAI(model="gpt-4o-mini")

def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 15"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NKbM1zfNcTYkwpi08Rmvq2gw', 'function': {'arguments': '{"a":3,"b":15}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 86, 'total_tokens': 104, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1ae3b419-011a-4848-b7b8-c55b0f7476a7-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 15}, 'id': 'call_NKbM1zfNcTYkwpi08Rmvq2gw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 18, 'total_tokens': 104, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

从上述调用结果上可以看出，如果大模型响应包含了 tool calls，会在 `.tool_calls` 属性上体现。

如果大模型响应了一个无效的 tool calls，则会在 `.invalid_tool_calls` 属性上体现。

可以使用 [output parsers](https://python.langchain.com/docs/how_to/#output-parsers) 解析 tool calls 结果。

In [4]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env",verbose=True)

llm = ChatOpenAI(model="gpt-4o-mini")

class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 15"

chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[multiply(a=3, b=15)]

当大模型响应 tool calls，agent 解析后会调用 tools，现在问题是如何把调用结果再返回给大模型

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env",verbose=True)

llm = ChatOpenAI(model="gpt-4o-mini")

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)
print(ai_msg.tool_calls)

messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

llm_with_tools.invoke(messages)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_LB1z4yKvvmNtHGzNao55HfnQ', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_gMupNuSKoDBsk2G0MNySTTRV', 'type': 'tool_call'}]


AIMessage(content='The result of \\(3 \\times 12\\) is \\(36\\), and the result of \\(11 + 49\\) is \\(60\\).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 153, 'total_tokens': 187, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None}, id='run-8f6f230b-bc72-4fc7-9e04-6ee3cb1e357f-0', usage_metadata={'input_tokens': 153, 'output_tokens': 34, 'total_tokens': 187, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})